In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from scipy.ndimage import filters
import imtools
%matplotlib inline

img_path = 'images/at_large.jpg'

In [ ]:
# Compute Average
def compute_average(image_list):
    """ Compute the average of a list of images """
    # check that all elements are the same size
    images = np.array([Image.open(image) for image in image_list], 'f') 
    average_img = images[0]
    if all(len(img) == len(images[0]) for img in image):
        for img in images[1:]:
            average_img += img
        average_img /= len(images)
        return array(average_img, 'uint8')
    else:
        return nil

In [ ]:
# Gaussian Blur Example
NUM_CHANNELS = 3

img = Image.open(img_path)

image_array = np.array(img)
filtered_image_array = np.zeros(image_array.shape)
theta = 8
for i in range(NUM_CHANNELS):
    filtered_image_array[:,:,i] = filters.gaussian_filter(image_array[:,:,i], theta)

filtered_image_array = np.uint8(filtered_image_array)    
    
plt.imshow(filtered_image_array)
plt.show()

In [ ]:
# Image Derivatives
# Describe how intensity changes over the image

# Calculate x and y derivate approximations with sobel filters
image_array = np.array(Image.open(img_path).convert('L'))
image_x = np.zeros(image_array.shape)

filters.sobel(image_array, 1, image_x)

image_y = np.zeros(image_array.shape)
filters.sobel(image_array, 0, image_y)

# Calculate gradient magnitude, which describes intensity change
magnitude = np.sqrt(image_x**2+image_y**2)

plt.gray();

plt.figure()
plt.imshow(image_array)

plt.figure()
plt.imshow(image_x)

plt.figure()
plt.imshow(image_y)

plt.figure()
plt.imshow(magnitude)

plt.show()


In [ ]:
# Using gaussian filters is more robust 
# and computes derivatives without respect to scale
img = np.array(Image.open(img_path).convert('L'))

sigma = 5
imx = np.zeros(img.shape)
filters.gaussian_filter(img, (sigma,sigma), (0,1), imx)

imy = np.zeros(img.shape)
filters.gaussian_filter(img, (sigma,sigma), (1,0), imy)

plt.gray();

plt.figure()
plt.imshow(imx)

plt.figure()
plt.imshow(imy)

In [ ]:
# Morphology Example
# Methods for measuring and analyzing basic shapes
from scipy.ndimage import measurements, morphology

img = np.array(Image.open('images/houses.png').convert('L'))

plt.gray();

plt.figure()
plt.imshow(img)

img = 1*(img<128)

plt.figure()
plt.imshow(img)

labels, num_objects = measurements.label(img)
print 'Number of objects: ', num_objects

plt.figure()
plt.imshow(labels)

# morphology - opening to separate objects better
# 9, 5 specifies neighbors to use for a given pixel, and then the number of iterations
# http://docs.scipy.org/doc/scipy/reference/ndimage.html

im_open = morphology.binary_opening(img,np.ones((9,5)),iterations=2)

labels_open, num_objects_open = measurements.label(im_open)
plt.figure()
plt.imshow(labels_open)

print 'Number of objects: ', num_objects_open




In [ ]:
# Image De-noising
# ROF Model - Smoother images while preserving edges and structures
def denoise(img, U_init, tolerance=0.1, tau=0.125, tv_weight=100):
    """ An implementation of the Rudin-Osher-Fatemi (ROF) denoising model
    using the numerical procedure presented in eq (11) A. Chambolle (2005).
    Input: noisy input image (grayscale), initial guess for U, weight of the TV-regularizing term, steplength, tolerance for stop criterion.
    Output: denoised and detextured image, texture residual. """
    m, n = img.shape
    #Init
    U = U_init
    Px = img
    Py = img
    error = 1
    
    while(error > tolerance):
        Uold = U
        
        # gradient of primal variable
        GradUx = np.roll(U, -1, axis=1)
        GradUy = np.roll(U, -1, axis=0)
        
        # update the dual varible
        PxNew = Px + (tau/tv_weight)*GradUx
        PyNew = Py + (tau/tv_weight)*GradUy
        NormNew = np.maximum(1,np.sqrt(PxNew**2+PyNew**2))
        
        # update of x-component (dual) 
        Px = PxNew/NormNew 
        
        # update of y-component (dual)
        Py = PyNew/NormNew 
        
        # update the primal variable
        RxPx = np.roll(Px,1,axis=1) # right x-translation of x-component
        RyPy = np.roll(Py,1,axis=0) # right y-translation of y-component

        DivP = (Px-RxPx)+(Py-RyPy) # divergence of the dual field. 
        U = img + tv_weight*DivP # update of the primal variable
        
        # update of error
        error = np.linalg.norm(U-Uold)/np.sqrt(n*m);
        
    return U, img-U
            
import scipy.misc

img = scipy.misc.lena()
noised_img = img + 30 * np.random.standard_normal(img.shape)

plt.figure()
plt.imshow(img)

plt.figure()
plt.imshow(noised_img)

U,T = denoise(noised_img, noised_img)

plt.figure()
plt.imshow(U)



